# 保存模型, 加载模型和保存参数

keras 保存成hdf5文件, 1,保存模型和参数;2, 只保存参数

1,保存模型和参数
     
     1) .save
     
     2).callback ModelCheckpoint

2, 只保存参数
     
     1) save_weights
     
     2) callback ModelCheckpoint save_weights_only=True

In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

4422102/4422102 [==============================] - 32s 7us/step
(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [3]:
# 标准化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(55000, -1))
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(5000, -1))

x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(10000, -1))

In [4]:
# 创建模型
model = keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax'),
])

model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [5]:
import os

logdir = './graph_def_and_weights'
if not os.path.exists(logdir):
    os.mkdir(logdir)
    
output_model_file = os.path.join(logdir, 'fashion_mnist_weight.h5')
callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file, 
                                    save_best_only=True,
                                    save_weights_only=True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)
]
history = model.fit(x_train_scaled, y_train, epochs=10, 
                   validation_data=(x_valid_scaled, y_valid),
                   callbacks=callbacks)

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4515 - accuracy: 0.8365 - val_loss: 0.3578 - val_accuracy: 0.8712
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3461 - accuracy: 0.8726 - val_loss: 0.3501 - val_accuracy: 0.8740
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3089 - accuracy: 0.8853 - val_loss: 0.3326 - val_accuracy: 0.8766
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2802 - accuracy: 0.8964 - val_loss: 0.3232 - val_accuracy: 0.8896
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2611 - accuracy: 0.9003 - val_loss: 0.3268 - val_accuracy: 0.8828
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2404 - accuracy: 0.9091 - val_loss: 0.3448 - val_accuracy: 0.8856
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2254 - accuracy: 0.9152 - val_loss: 0.3319 - val_accuracy:

In [6]:
# 保存模型
output_model_file2 = os.path.join(logdir, 'fashion_mnist_model.h5')
model.save(output_model_file2)

In [7]:
model.evaluate(x_valid_scaled, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 0.3711 - accuracy: 0.8854


[0.3710867464542389, 0.8853999972343445]

In [8]:
# 加载模型
model2 = keras.models.load_model(output_model_file2)

In [9]:
model2.evaluate(x_valid_scaled,y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 0.3711 - accuracy: 0.8854


[0.3710867464542389, 0.8853999972343445]

In [10]:
# 另一种保存参数的方法
model.save_weights(os.path.join(logdir, 'fashin_mnist_weights_2.h5'))

# 加载参数

In [11]:
# 只加载参数
import os

logdir = './graph_def_and_weights'
if not os.path.exists(logdir):
    os.mkdir(logdir)
    
output_model_file = os.path.join(logdir, 'fashion_mnist_weight.h5')

model.load_weights(output_model_file)

In [12]:
model.evaluate(x_valid_scaled, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 0.3232 - accuracy: 0.8896


[0.32321834564208984, 0.8895999789237976]

# keras_to_savedmodel

In [13]:
# 把keras模型保存成savedmodel格式
tf.saved_model.save(model, './keras_saved_model')

INFO:tensorflow:Assets written to: ./keras_saved_model\assets


In [14]:
!saved_model_cli show --dir ./keras_saved_model --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dense_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 784)
        name: serving_default_dense_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_3'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Concrete Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          inputs: TensorS

In [15]:
!saved_model_cli show --dir ./keras_saved_model --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['dense_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 784)
      name: serving_default_dense_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_3'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [16]:
# 加载savedmodel模型
loaded_saved_model = tf.saved_model.load('./keras_saved_model')

In [17]:
loaded_saved_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x223b6422ca0>

In [18]:
list(loaded_saved_model.signatures.keys())

['serving_default']

In [19]:
inference = loaded_saved_model.signatures['serving_default']
print(inference)

ConcreteFunction signature_wrapper(*, dense_input)
  Args:
    dense_input: float32 Tensor, shape=(None, 784)
  Returns:
    {'dense_3': <1>}
      <1>: float32 Tensor, shape=(None, 10)


In [20]:
# 查看一下输入
print(inference.structured_outputs)

{'dense_3': TensorSpec(shape=(None, 10), dtype=tf.float32, name='dense_3')}


In [21]:
print(inference.structured_input_signature)

((), {'dense_input': TensorSpec(shape=(None, 784), dtype=tf.float32, name='dense_input')})


In [22]:
# 使用saved_model
results = inference(tf.constant(x_test_scaled[0:1]))
print(results)

{'dense_3': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[6.0051505e-08, 7.4299655e-12, 1.6064561e-09, 8.2079288e-11,
        3.2423002e-11, 7.1292911e-03, 7.3568701e-10, 2.6837764e-02,
        2.7522844e-09, 9.6603286e-01]], dtype=float32)>}


In [23]:
results['dense_3']

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[6.0051505e-08, 7.4299655e-12, 1.6064561e-09, 8.2079288e-11,
        3.2423002e-11, 7.1292911e-03, 7.3568701e-10, 2.6837764e-02,
        2.7522844e-09, 9.6603286e-01]], dtype=float32)>

# to_tflite

In [24]:
# 从文件加载
loaded_keras_model = keras.models.load_model('./graph_def_and_weights/fashion_mnist_model.h5')

In [25]:
# (-1, 784)
loaded_keras_model(np.ones((1, 784)))

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[2.6461650e-02, 3.2110941e-02, 1.2605472e-01, 6.0098335e-02,
        5.3037982e-02, 1.9675593e-03, 6.1453022e-02, 5.9859216e-04,
        6.3768595e-01, 5.3128914e-04]], dtype=float32)>

In [28]:
np.ones((1, 784))

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 

## keras to tflite

In [29]:
# lite 精简版模型
# 创建转化器
keras_to_tflite_converter = tf.lite.TFLiteConverter.from_keras_model(loaded_keras_model)
keras_to_tflite_converter

In [30]:
# 执行转化
keras_tflite = keras_to_tflite_converter.convert()

INFO:tensorflow:Assets written to: C:\Users\ADMINI~1\AppData\Local\Temp\tmp838guhaw\assets


In [31]:
# 写入指定的文件
import os

if not os.path.exists('./tflite_models'):
    os.mkdir('./tflite_models')
    
with open('./tflite_models/keras_tflite', 'wb') as f:
    f.write(keras_tflite)

## concrete function to tflite

In [32]:
# 把keras模型转化成concrete function
run_model = tf.function(lambda x: loaded_keras_model(x))
keras_concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(loaded_keras_model.inputs[0].shape,
                 loaded_keras_model.inputs[0].dtype)
)

In [33]:
concrete_func_to_tflite = tf.lite.TFLiteConverter.from_concrete_functions([keras_concrete_func])
concrete_func_tflite = concrete_func_to_tflite.convert()

In [34]:
with open('./tflite_models/concrete_func_tflite', 'wb') as f:
    f.write(concrete_func_tflite)

## saved_model to tflite

In [35]:
saved_model_to_tflite_converter = tf.lite.TFLiteConverter.from_saved_model('./keras_saved_model/')
saved_model_tflite = saved_model_to_tflite_converter.convert()

In [36]:
with open('./tflite_models/saved_model_tflite', 'wb') as f:
    f.write(saved_model_tflite)

# tflite_interpreter

In [37]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [38]:
with open('./tflite_models/concrete_func_tflite', 'rb') as f:
    concrete_func_tflite = f.read()

In [39]:
# 创建interpreter
interpreter = tf.lite.Interpreter(model_content=concrete_func_tflite)
# 分配内存
interpreter.allocate_tensors()

In [40]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'x', 'index': 0, 'shape': array([  1, 784]), 'shape_signature': array([ -1, 784]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'Identity', 'index': 13, 'shape': array([ 1, 10]), 'shape_signature': array([-1, 10]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [41]:
input_data = tf.constant(np.ones(input_details[0]['shape'], dtype=np.float32))
# 传入预测数据
interpreter.set_tensor(input_details[0]['index'], input_data)

# 执行预测
interpreter.invoke()

# 获取输出
output_results = interpreter.get_tensor(output_details[0]['index'])
print(output_results)

[[2.6461685e-02 3.2110948e-02 1.2605478e-01 6.0098335e-02 5.3038076e-02
  1.9675621e-03 6.1453070e-02 5.9859303e-04 6.3768566e-01 5.3129060e-04]]


# to_quantized_tflite

## keras to tflite

In [43]:
# lite 精简版模型
# 创建转化器
keras_to_tflite_converter = tf.lite.TFLiteConverter.from_keras_model(loaded_keras_model)
keras_to_tflite_converter

In [45]:
# 给converter添加量化
# 把32位浮点数变成8位整数
keras_to_tflite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
# 执行转化
keras_tflite = keras_to_tflite_converter.convert()

INFO:tensorflow:Assets written to: C:\Users\ADMINI~1\AppData\Local\Temp\tmp6j_e0r_k\assets


INFO:tensorflow:Assets written to: C:\Users\ADMINI~1\AppData\Local\Temp\tmp6j_e0r_k\assets


In [46]:
# 写入指定的文件
import os

if not os.path.exists('./tflite_models'):
    os.mkdir('./tflite_models')
    
with open('./tflite_models/quantized_keras_tflite', 'wb') as f:
    f.write(keras_tflite)

## concrete function to tflite

In [47]:
# 把keras模型转化成concrete function
run_model = tf.function(lambda x: loaded_keras_model(x))
keras_concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(loaded_keras_model.inputs[0].shape,
                 loaded_keras_model.inputs[0].dtype)
    )

In [49]:
concrete_func_to_tflite_converter = tf.lite.TFLiteConverter.from_concrete_functions([keras_concrete_func])
concrete_func_to_tflite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
concrete_func_tflite = concrete_func_to_tflite_converter.convert()

In [50]:
with open('./tflite_models/quantized_concrete_func_tflite', 'wb') as f:
    f.write(concrete_func_tflite)

## saved_model to tflite

In [52]:
saved_model_to_tflite_converter = tf.lite.TFLiteConverter.from_saved_model('./keras_saved_model/')
saved_model_to_tflite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
saved_model_tflite = saved_model_to_tflite_converter.convert()

In [53]:
with open('./tflite_models/quantized_saved_model_tflite', 'wb') as f:
    f.write(saved_model_tflite)